In [158]:
import pandas as pd
import numpy as np


#### Reading data

In [159]:
CA_data = pd.read_csv('worksheet_graphs - Sheet1.csv')

columns = ['Weather Vehicle 1', 'Lighting Vehicle 1', 'Roadway Surface Vehicle 1', 'Movement Preceding Collision Vehicle 1']

### Preprocess Data

In [160]:
# dropping columns with null data
for col in columns:
    print("\n")
    CA_data.drop(CA_data[CA_data[col] == 'Not Available'].index, inplace=True)
    print(CA_data[col].value_counts())





Clear             265
Cloudy             29
Raining            10
Fog/Visibility      3
Wind                3
Slippery            1
Dark                1
Name: Weather Vehicle 1, dtype: int64


Daylight                                227
Dark w Street-lights                     76
Dusk/Dawn                                 5
No unusual conditions                     2
Dark w Non-functioning Street-lights      2
Name: Lighting Vehicle 1, dtype: int64


Dry                    286
Wet                     14
Slippery                 8
Other                    1
Proceeding Straight      1
Name: Roadway Surface Vehicle 1, dtype: int64


Stopped                                130
Proceeding Straight                     91
Slowing/Stopping                        19
Making Right Turn                       15
Making Left Turn                        12
Parking                                 10
Backing                                  8
Changing Lanes                           5
Other           

### Grouping data

#### Column Weather Vehicle 1
We are grouping Fog/visiblity with Dark because they are visual impariements\
We are grouping slippery, wind with raining because when it rains, it usally reults in slipperiness and sometimes wind\

#### Lighting Vehicle 1
Grouping no unusla condition and daylight b/c they basically the same\
Grouping Dusk/dawn with dark with street lights b/c they are relatively similar conditions\

#### Movement Preceding Collision Vehicle 1
Wet roads = slippery roads\
Dropped other and proceeding straight b/c they dont provide much value\



In [161]:
# grouping less frequent data 
CA_data['Weather Vehicle 1'].replace(['Fog/Visibility', 'Dark'], 'Fog/Visibility/Dark', inplace=True)
CA_data['Weather Vehicle 1'].replace(['Slippery', 'Wind'], 'Raining', inplace=True)

CA_data['Lighting Vehicle 1'].replace(['No unusual condition'], 'Daylight', inplace=True)
CA_data['Lighting Vehicle 1'].replace(['Dusk/Dawn'], 'Dark w Street-lights', inplace=True)


CA_data['Roadway Surface Vehicle 1'].replace(['Slippery'], 'Wet', inplace=True)
CA_data.drop(CA_data[(CA_data['Roadway Surface Vehicle 1'] == 'Other') | (CA_data['Roadway Surface Vehicle 1'] == 'Proceeding Straight')].index, inplace=True)

CA_data['Movement Preceding Collision Vehicle 1'].replace(['Making Right Turn', 'Making Left Turn', 'Making U turn', 'Making Right Turn, Slowing/Stopping'], 'Turning', inplace=True)
CA_data['Movement Preceding Collision Vehicle 1'].replace(['Changing Lanes', 'Entering Traffic', 'Entrering Traffic','Xing into opposing lane', 'Passing Other Vehicle'], 'Highway movement', inplace=True)
CA_data['Movement Preceding Collision Vehicle 1'].replace(['Slowing/Stopping', 'Stopped in Traffic', 'Parked', 'Stopped, Merging'], 'Stopped', inplace=True)
CA_data['Movement Preceding Collision Vehicle 1'].replace(['Parking Manuerver'], 'Parking', inplace=True)
CA_data.drop(CA_data[(CA_data['Movement Preceding Collision Vehicle 1'] == 'Other')].index, inplace=True)


for col in columns:
    print()
    print(CA_data[col].value_counts())


Clear                  261
Cloudy                  28
Raining                 11
Fog/Visibility/Dark      3
Name: Weather Vehicle 1, dtype: int64

Daylight                224
Dark w Street-lights     79
Name: Lighting Vehicle 1, dtype: int64

Dry    283
Wet     20
Name: Roadway Surface Vehicle 1, dtype: int64

Stopped                151
Proceeding Straight     91
Turning                 29
Parking                 11
Highway movement        10
Backing                  8
Parking Manuever         3
Name: Movement Preceding Collision Vehicle 1, dtype: int64


In [162]:
# one hot encode our new columns
encoded_CA_data = pd.get_dummies(CA_data, columns=columns, prefix='encoded_')
encoded_columns = encoded_CA_data.columns[encoded_CA_data.columns.str.startswith('encoded_')]



Preprocess outputs

In [163]:
from sklearn.preprocessing import LabelEncoder

encoded_CA_data.drop(encoded_CA_data[(encoded_CA_data['Vehicle Damage'] == 'Not Available')].index, inplace=True)



label_encoder = LabelEncoder()
encoded_CA_data['Vehicle Damage'] = label_encoder.fit_transform(encoded_CA_data['Vehicle Damage'])

print(encoded_CA_data['Vehicle Damage'].value_counts())


1    217
3     53
2     49
0      8
Name: Vehicle Damage, dtype: int64


In [164]:
# Split data into training and testing
from sklearn.model_selection import train_test_split

X = encoded_CA_data[encoded_columns]
y = encoded_CA_data['Vehicle Damage']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)   



In [165]:
y.value_counts()

1    217
3     53
2     49
0      8
Name: Vehicle Damage, dtype: int64

### SVM

In [169]:
from sklearn import svm
from sklearn.model_selection import cross_val_score


clf = svm.SVC(kernel='linear')
scores = cross_val_score(clf, X, y, cv=7)

scores




array([0.65957447, 0.63829787, 0.74468085, 0.82978723, 0.80851064,
       0.76086957, 0.67391304])

### Neural network

In [170]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class FNN(nn.Module):
    def __init__(self, input_channels, output_channels):

        super(FNN, self).__init__()
        self.fc1 = nn.Linear(input_channels, 6)
        self.fc2 = nn.Linear(6, 4)
        self.fc3 = nn.Linear(4, output_channels)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [171]:
X_train_np = X_train.values
X_test_np = X_test.values



X_train_tensor = torch.tensor(X_train_np)
y_train_tensor = torch.tensor(y_train)
X_test_tensor = torch.tensor(X_test_np)
y_test_tensor = torch.tensor(y_test)


KeyError: 3

In [ ]:
# model
model = FNN(X_train_tensor.shape[1], y.value_counts().shape[0]-1)
print(X_train_tensor.shape[1], y.value_counts().shape[0]-1)
device = 'cuda'
model.to(device)

# hyper parameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 10


15 3


In [ ]:
from sklearn.metrics import accuracy_score


for epoch in range(epochs):
    inputs, targets = X_train_tensor.to(device), y_train_tensor.to(device)
    optimizer.zero_grad()  # Zero the gradients
    outputs = model(inputs)  # Forward pass
    loss = criterion(outputs, targets)  # Compute the loss
    loss.backward()  # Backward pass (compute gradients)
    optimizer.step()  # Update the weights

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}')



RuntimeError: mat1 and mat2 must have the same dtype, but got Byte and Float

In [ ]:
# Evaluation on test data

model.toeval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    accuracy = accuracy_score(y_test_tensor.numpy(), predicted.numpy())
    print(f'Test Accuracy: {accuracy}')